<a href="https://colab.research.google.com/github/datajoedata/e-Commerce-Sales-Analysis-Target-Brazil/blob/main/pipeline_silver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I - Imports e montagem do drive

In [1]:
from google.colab import drive
import os
import pandas as pd
from zipfile import ZipFile

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# II - Funções Auxiliares


In [3]:
# Descompacta arquivos em uma pasta especificada
def descompactar_arquivos(zip_path, extrair_para='/content/'):
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extrair_para)
    print(f"Arquivos descompactados em: {extrair_para}")

# Lista arquivos descompactados
def listar_arquivos(caminho):
    arquivos = os.listdir(caminho)
    print("Arquivos descompactados:", arquivos)
    return arquivos

# Carrega arquivos CSV em DataFrames
def carregar_csv(caminho_arquivo, nome_arquivo):
    return pd.read_csv(f"{caminho_arquivo}/{nome_arquivo}")

# Cria uma pasta, se ela não existir
def criar_pasta(caminho):
    if not os.path.exists(caminho):
        os.makedirs(caminho)

# Remover uma pasta e seus arquivos
def remover_pasta(caminho):
    for arquivo in os.listdir(caminho):
        os.remove(os.path.join(caminho, arquivo))
    os.rmdir(caminho)
    print(f"Pasta '{caminho}' removida.")

# III - Funções de Limpeza


In [4]:
# Remove duplicatas em uma coluna específica
def remover_duplicatas(df, coluna):
    linhas_antes = len(df)
    df_sem_duplicatas = df.drop_duplicates(subset=coluna, keep='first')
    linhas_removidas = linhas_antes - len(df_sem_duplicatas)
    print(f'Total de linhas removidas com base na coluna "{coluna}": {linhas_removidas}')
    return df_sem_duplicatas

# Limpa colunas string (remover espaços extras)
def limpar_strings(df):
    return df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Converte colunas para tipos específicos


def converter_tipo(df, colunas, tipo, errors='raise'):
    for coluna in colunas:
        df[coluna] = df[coluna].astype(tipo, errors=errors)
    return df

# Converte colunas para datetime

def converter_para_datetime(df, colunas):
    for coluna in colunas:
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    return df

# IV - Funções de Salvamento

In [5]:
# Função para salvar DataFrames em formato parquet na pasta especificada
def salvar_em_parquet(df, nome_arquivo, caminho_silver):
    criar_pasta(caminho_silver)
    caminho_completo = f"{caminho_silver}/{nome_arquivo}.parquet"
    df.to_parquet(caminho_completo, index=False)
    print(f"Arquivo salvo em Parquet: {caminho_completo}")

# V - Pipeline


In [6]:
# Processar cada DataFrame (limpeza específica de cada um)
def processar_clientes(df):
    df = remover_duplicatas(df, 'customer_unique_id')
    df = converter_tipo(df, ['customer_zip_code_prefix'], 'str')
    return limpar_strings(df)

def processar_vendedores(df):
    df = remover_duplicatas(df, 'seller_id')
    df = converter_tipo(df, ['seller_zip_code_prefix'], 'str')
    return limpar_strings(df)

def processar_itens_pedidos(df):
    df = remover_duplicatas(df, 'order_id')
    df = converter_tipo(df, ['order_item_id'], 'str')
    return converter_para_datetime(df, ['shipping_limit_date'])

def processar_geolocalizacao(df):
    df = converter_tipo(df, ['geolocation_zip_code_prefix'], 'str')
    return limpar_strings(df)

def processar_pagamentos(df):
    df = remover_duplicatas(df, 'order_id')
    df = limpar_strings(df)
    df['payment_type'] = df['payment_type'].str.lower()
    return df

def processar_pedidos(df):
    df = remover_duplicatas(df, 'order_id')
    df = converter_para_datetime(df, [
        'order_purchase_timestamp', 'order_approved_at',
        'order_delivered_carrier_date', 'order_delivered_customer_date',
        'order_estimated_delivery_date'
    ])
    return limpar_strings(df)

def processar_produtos(df):
    df = remover_duplicatas(df, 'product_id')
    df.rename(columns={'product category': 'product_category'}, inplace=True)
    df['product_category'] = df['product_category'].str.lower().str.strip()
    return df



# VI - Função Principal


In [7]:
def executar_pipeline(zip_path, caminho_bronze, caminho_silver):
    print("\n=== Início da Pipeline ===\n")

    # Cria pasta temporária de bronze
    criar_pasta(caminho_bronze)

    # Descompacta arquivos na pasta bronze e lista arquivos
    descompactar_arquivos(zip_path, caminho_bronze)
    listar_arquivos(caminho_bronze)

    # Carrega DataFrames e aplica processamento específico
    print("\n=== Processamento dos DataFrames ===")
    clientes_df = processar_clientes(carregar_csv(caminho_bronze, 'customers.csv'))
    vendedores_df = processar_vendedores(carregar_csv(caminho_bronze, 'sellers.csv'))
    itens_pedidos_df = processar_itens_pedidos(carregar_csv(caminho_bronze, 'order_items.csv'))
    geolocalizacao_df = processar_geolocalizacao(carregar_csv(caminho_bronze, 'geolocation.csv'))
    pagamentos_df = processar_pagamentos(carregar_csv(caminho_bronze, 'payments.csv'))
    pedidos_df = processar_pedidos(carregar_csv(caminho_bronze, 'orders.csv'))
    produtos_df = processar_produtos(carregar_csv(caminho_bronze, 'products.csv'))

    # Salva DataFrames processados em formato Parquet na pasta silver com sufixo
    print("\n=== Salvando DataFrames em Parquet ===")
    salvar_em_parquet(clientes_df, 'clientes', caminho_silver)
    salvar_em_parquet(vendedores_df, 'vendedores', caminho_silver)
    salvar_em_parquet(itens_pedidos_df, 'itens_pedidos', caminho_silver)
    salvar_em_parquet(geolocalizacao_df, 'geolocalizacao', caminho_silver)
    salvar_em_parquet(pagamentos_df, 'pagamentos', caminho_silver)
    salvar_em_parquet(pedidos_df, 'pedidos', caminho_silver)
    salvar_em_parquet(produtos_df, 'produtos', caminho_silver)

    # Remove a pasta temporária de bronze e seus arquivos
    remover_pasta(caminho_bronze)

    print("\n=== Pipeline Finalizada ===")

# VII - Execução da Pipeline

In [8]:
zip_path = '/content/drive/MyDrive/Analise_de_Vendas_E-commerce/archive (2).zip'
caminho_bronze = '/content/drive/MyDrive/Analise_de_Vendas_E-commerce/bronze-temp'
caminho_silver = '/content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver'

# Executa a pipeline completa
executar_pipeline(zip_path, caminho_bronze, caminho_silver)


=== Início da Pipeline ===

Arquivos descompactados em: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/bronze-temp
Arquivos descompactados: ['customers.csv', 'geolocation.csv', 'order_items.csv', 'orders.csv', 'payments.csv', 'products.csv', 'sellers.csv']

=== Processamento dos DataFrames ===
Total de linhas removidas com base na coluna "customer_unique_id": 3345
Total de linhas removidas com base na coluna "seller_id": 0


<ipython-input-4-f4de419b5123>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = df[coluna].astype(tipo, errors=errors)


Total de linhas removidas com base na coluna "order_id": 13984


<ipython-input-4-f4de419b5123>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = df[coluna].astype(tipo, errors=errors)
<ipython-input-4-f4de419b5123>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = pd.to_datetime(df[coluna], errors='coerce')


Total de linhas removidas com base na coluna "order_id": 4446
Total de linhas removidas com base na coluna "order_id": 0
Total de linhas removidas com base na coluna "product_id": 0

=== Salvando DataFrames em Parquet ===
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/clientes.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/vendedores.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/itens_pedidos.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/geolocalizacao.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/pagamentos.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/pedidos.parquet
Arquivo salvo em Parquet: /content/drive/MyDrive/Analise_de_Vendas_E-commerce/silver/produtos.parquet
Pasta '/content/drive/MyDrive/Analise_de_Vendas_E-